In [ ]:
import warnings
warnings.filterwarnings("ignore")

import argparse
import logging
import os
import sys
import time

from collections import defaultdict, OrderedDict

import matplotlib
import numpy as np
import soundfile as sf
import torch
import yaml

from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

import sys
sys.path.append("../")

import seq2seq_vc
import seq2seq_vc.models
import seq2seq_vc.losses
import seq2seq_vc.trainers
import seq2seq_vc.collaters

# from seq2seq_vc.datasets import ParallelVCMelDataset
from torch.utils.data import Dataset

from seq2seq_vc.utils import read_hdf5
from seq2seq_vc.utils.types import str_or_none
# from seq2seq_vc.vocoder import Vocoder
# from seq2seq_vc.vocoder.s3prl_feat2wav import S3PRL_Feat2Wav
# from seq2seq_vc.vocoder.griffin_lim import Spectrogram2Waveform
# from seq2seq_vc.vocoder.encodec import EnCodec_decoder

# set to avoid matplotlib error in CLI environment
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt

from seq2seq_vc.schedulers.warmup_lr import WarmupLR

scheduler_classes = dict(warmuplr=WarmupLR)

class Dict2Obj(object):
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

import joblib
import glob
datasplit = list(np.load("./data_split_ARCTIC.npy", allow_pickle=True))

class ParallelVCMelDataset(Dataset):
    def __init__(self, dataset_dir, speakers, src_spk, trg_spk, datasplit, scaler, mode="train"):
        modefiles = datasplit[["train", "valid", "test"].index(mode)]
        filenames = [os.path.basename(a)[:-4] for a in glob.glob(dataset_dir+f"{speakers[0]}/mel/*")]
        filenames.sort()
        files = []
        for fn in filenames:
            if fn in modefiles:
                exist = True
                for spk in speakers[1:]:
                    if not(os.path.exists(dataset_dir + f"{spk}/mel/{fn}.npy")):
                        exist = False
                        break
                if exist:
                    files += [fn]
        data = {}
        for spk in speakers:
            data[spk] = [dataset_dir + f"{spk}/mel/{fn}.npy" for fn in files]
            
        self.data = data
        self.src_spk = src_spk
        self.trg_spk = trg_spk
        self.scaler = scaler
        
    def __len__(self):
        return len(self.data[self.src_spk])
    def __getitem__(self, idx):
        items = {}
        items["src_feat"] = self.scaler.transform(np.load(self.data[self.src_spk][idx]).T)
        items["trg_feat"] = self.scaler.transform(np.load(self.data[self.trg_spk][idx]).T)
        
        return items

In [ ]:
# Dataset Variables
dataset_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
speakers = ["HKK", "TNI"]
src_spk = "HKK"
trg_spk = "TNI"

scaler_filename = f"ckpts/scalers/HKK_TNI.save"
scaler = joblib.load(scaler_filename)

In [ ]:
args = {}
args["rank"] = 0
args["outdir"] = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts/HKK_TNI/"
# args.config_path = "./../egs/l2-arctic/cascade/conf/vtn.tts_pt.v1.yaml"
args["config_path"] = "./../egs/arctic/vc1/conf/vtn.v1.melmel.yaml"
args["init_checkpoint"] = ""
args["resume"] = ""
args["distributed"] = False
args = Dict2Obj(args)

# load main config
with open(args.config_path) as f:
    config = yaml.load(f, Loader=yaml.Loader)
config.update(vars(args))

# Customization
config["model_params"]["idim"] = 100
config["model_params"]["odim"] = 100

In [ ]:
device = torch.device("cuda")
torch.backends.cudnn.benchmark = True
torch.cuda.set_device(args.rank)
if not os.path.exists(args.outdir):
    os.makedirs(args.outdir)
    
### Dataset Preparation ###
dataset = {
    "train": ParallelVCMelDataset(dataset_dir, speakers, src_spk, trg_spk, datasplit, scaler, "train"),
    "dev": ParallelVCMelDataset(dataset_dir, speakers, src_spk, trg_spk, datasplit, scaler, "valid"),
}

collater_class = getattr(
    seq2seq_vc.collaters,
    config.get("collater_type", "ARVCCollater"),
)
collater = collater_class()

sampler = {"train": None, "dev": None}
data_loader = {
    "train": DataLoader(
        dataset=dataset["train"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["train"],
        pin_memory=config["pin_memory"],
    ),
    "dev": DataLoader(
        dataset=dataset["dev"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["dev"],
        pin_memory=config["pin_memory"],
    ),
}

### Model Preparation ###
model_class = getattr(
    seq2seq_vc.models,
    config.get("model_type", "VTN"),
)
model = model_class(**config["model_params"]).to(device)

if config.get("criterions", None):
    criterion = {
        criterion_class: getattr(seq2seq_vc.losses, criterion_class)(
            **criterion_paramaters
        )
        for criterion_class, criterion_paramaters in config["criterions"].items()
    }
else:
    raise ValueError("Please specify criterions in the config file.")

### optimizers and schedulers ###
optimizer_class = getattr(
    torch.optim,
    # keep compatibility
    config.get("optimizer_type", "Adam"),
)
optimizer = optimizer_class(
    model.parameters(),
    **config["optimizer_params"],
)
scheduler_class = scheduler_classes.get(config.get("scheduler_type", "warmuplr"))
scheduler = scheduler_class(
    optimizer=optimizer,
    **config["scheduler_params"],
)

### define trainer ###
trainer_class = getattr(
    seq2seq_vc.trainers,
    config.get("trainer_type", "ARVCTrainer"),
)
trainer = trainer_class(
    steps=0,
    epochs=0,
    data_loader=data_loader,
    sampler=sampler,
    model=model,
    vocoder=None,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    config=config,
    device=device,
)

# load pretrained parameters from checkpoint
if len(args.init_checkpoint) != 0:
    trainer.load_trained_modules(
        args.init_checkpoint, init_mods=config["init-mods"]
    )

# resume from checkpoint
if len(args.resume) != 0:
    trainer.load_checkpoint(args.resume)

# freeze modules if necessary
if config.get("freeze-mods", None) is not None:
    assert type(config["freeze-mods"]) is list
    trainer.freeze_modules(config["freeze-mods"])

NameError: name 'speakers' is not defined

In [ ]:
try:
    trainer.run()
finally:
    trainer.save_checkpoint(
        os.path.join(config["outdir"], f"checkpoint-{trainer.steps}steps.pkl")
    )
    logging.info(f"Successfully saved checkpoint @ {trainer.steps}steps.")

[train]:   0%|          | 10/50000 [00:23<26:35:47,  1.92s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[train]:   0%|          | 20/50000 [00:37<16:43:28,  1.20s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[train]:   0%|          | 30/50000 [00:50<14:04:19,  1.01s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[train]:   0%|          | 40/50000 [00:59<12:11:58,  1.14it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork.

- Output of ParallelVCMelDataset
    - src_feat
    - trg_feat
    - dp_input # duration predictor's input
    - duration # no need
    - utt_id